<a href="https://colab.research.google.com/github/sanjib7777/-Hyperspectral-Image-Classification/blob/main/indian_pipes_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from torch.utils.data import Dataset, DataLoader

In [3]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# 1. Load and preprocess Indian Pines dataset
def load_indian_pines():
    data = sio.loadmat('Indian_pines_corrected.mat')['indian_pines_corrected']
    labels = sio.loadmat('Indian_pines_gt.mat')['indian_pines_gt']
    return data, labels

def preprocess(data):
    data = data.astype(np.float32)
    data -= np.min(data)
    data /= np.max(data)
    return data

def extract_patches(data, labels, patch_size=5):
    margin = patch_size // 2
    padded_data = np.pad(data, ((margin, margin), (margin, margin), (0, 0)), mode='constant')
    h, w, c = data.shape
    patches, targets, indices = [], [], []
    for i in range(h):
        for j in range(w):
            if labels[i, j] == 0:
                continue
            patch = padded_data[i:i+patch_size, j:j+patch_size, :]
            patches.append(patch)
            targets.append(labels[i, j])
            indices.append((i, j))
    return np.array(patches), np.array(targets), np.array(indices)

# 2. Model components
class SpectralSpatialCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv3d = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv3d(8, 16, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 5, 5, in_channels)
            dummy_output = self.conv3d(dummy_input)
            self.flattened_size = dummy_output.view(1, -1).shape[1]
        self.fc = nn.Linear(self.flattened_size, 128)

    def forward(self, x):
        x = x.unsqueeze(1)  # add channel dim
        x = self.conv3d(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class GCNBranch(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, 128)
        self.fc = nn.Linear(128, 128)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return x

class DualBranchFusion(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.branch1 = SpectralSpatialCNN(in_channels, num_classes)
        self.branch2 = GCNBranch(in_channels, num_classes)
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x_img, x_graph_all, edge_index, batch_indices):
        f1 = self.branch1(x_img)
        f2_all = self.branch2(x_graph_all, edge_index)
        f2 = f2_all[batch_indices]
        fused = torch.cat((f1, f2), dim=1)
        out = self.classifier(fused)
        return out

# 3. Metrics
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred)
    return acc, f1, kappa


In [5]:
# 4. Dataset class
class IndianPinesDataset(Dataset):
    def __init__(self, x_img, x_graph, y):
        self.x_img = x_img
        self.x_graph = x_graph
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x_img[idx], self.x_graph[idx], self.y[idx]

In [6]:
# 5. Main
if __name__ == '__main__':
    data, labels = load_indian_pines()
    data = preprocess(data)
    patches, targets, indices = extract_patches(data, labels)

    patches = patches.reshape((patches.shape[0], -1, patches.shape[-1]))
    le = LabelEncoder()
    targets = le.fit_transform(targets)

    scaler = StandardScaler()
    x_graph = torch.tensor(scaler.fit_transform(patches[:, patches.shape[1]//2]), dtype=torch.float32)

    adj = kneighbors_graph(x_graph.numpy(), n_neighbors=8, mode='connectivity', include_self=False)
    edge_index_np = dense_to_sparse(torch.tensor(adj.toarray(), dtype=torch.float))[0]
    edge_index = edge_index_np.long()

    num_samples = patches.shape[0]
    x_img = torch.tensor(patches[:, :, :].reshape(num_samples, 5, 5, data.shape[2]), dtype=torch.float32)
    y = torch.tensor(targets, dtype=torch.long)

    indices = np.arange(num_samples)
    train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

    x_train, x_test = x_img[train_idx], x_img[test_idx]
    xg_train, xg_test = x_graph[train_idx], x_graph[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    train_dataset = IndianPinesDataset(x_train, xg_train, y_train)
    test_dataset = IndianPinesDataset(x_test, xg_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    train_mask = torch.zeros(num_samples, dtype=torch.bool)
    train_mask[train_idx] = True
    edge_mask = train_mask[edge_index[0]] & train_mask[edge_index[1]]
    edge_index_train = edge_index[:, edge_mask]

    model = DualBranchFusion(in_channels=data.shape[2], num_classes=len(np.unique(targets))).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss()

    # Training
    model.train()
    for epoch in range(100):
        epoch_loss = 0.0
        x_graph_all = x_graph.to(device)
        edge_index_train = edge_index_train.to(device)

        for x_img_batch, xg_batch, y_batch in train_loader:
            x_img_batch = x_img_batch.to(device)
            xg_batch = xg_batch.to(device)
            y_batch = y_batch.to(device)

            batch_indices = (xg_batch[:, None] == x_graph_all).all(dim=2).nonzero(as_tuple=True)[1]

            optimizer.zero_grad()
            outputs = model(x_img_batch, x_graph_all, edge_index_train, batch_indices)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")


Epoch 1, Loss: 3.3884
Epoch 2, Loss: 1.3339
Epoch 3, Loss: 1.2687
Epoch 4, Loss: 1.0368
Epoch 5, Loss: 0.9284
Epoch 6, Loss: 0.7507
Epoch 7, Loss: 0.6680
Epoch 8, Loss: 0.6975
Epoch 9, Loss: 0.5555
Epoch 10, Loss: 0.5355
Epoch 11, Loss: 0.4809
Epoch 12, Loss: 0.4624
Epoch 13, Loss: 0.4281
Epoch 14, Loss: 0.4154
Epoch 15, Loss: 0.4234
Epoch 16, Loss: 0.4067
Epoch 17, Loss: 0.3808
Epoch 18, Loss: 0.3673
Epoch 19, Loss: 0.3392
Epoch 20, Loss: 0.3423
Epoch 21, Loss: 0.3401
Epoch 22, Loss: 0.3322
Epoch 23, Loss: 0.3110
Epoch 24, Loss: 0.3301
Epoch 25, Loss: 0.2858
Epoch 26, Loss: 0.3027
Epoch 27, Loss: 0.2775
Epoch 28, Loss: 0.2878
Epoch 29, Loss: 0.2610
Epoch 30, Loss: 0.2684
Epoch 31, Loss: 0.2542
Epoch 32, Loss: 0.2543
Epoch 33, Loss: 0.2583
Epoch 34, Loss: 0.2415
Epoch 35, Loss: 0.2304
Epoch 36, Loss: 0.2165
Epoch 37, Loss: 0.2504
Epoch 38, Loss: 0.2521
Epoch 39, Loss: 0.2516
Epoch 40, Loss: 0.2180
Epoch 41, Loss: 0.2015
Epoch 42, Loss: 0.2117
Epoch 43, Loss: 0.1821
Epoch 44, Loss: 0.21

In [9]:
    # Evaluation
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        x_graph_all = x_graph.to(device)
        edge_index_train = edge_index_train.to(device)

        for x_img_batch, xg_batch, y_batch in test_loader:
            x_img_batch = x_img_batch.to(device)
            xg_batch = xg_batch.to(device)
            y_batch = y_batch.to(device)

            batch_indices = (xg_batch[:, None] == x_graph_all).all(dim=2).nonzero(as_tuple=True)[1]
            outputs = model(x_img_batch, x_graph_all, edge_index_train, batch_indices)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)

    oa = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred)

    print(f"Overall Accuracy (OA): {oa:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")

Overall Accuracy (OA): 0.9145
F1 Score (Macro): 0.9236
Cohen's Kappa: 0.9026


In [10]:
    # Training Set Evaluation
    model.eval()
    all_train_preds, all_train_labels = [], []

    with torch.no_grad():
        x_graph_all = x_graph.to(device)
        edge_index_train = edge_index_train.to(device)

        for x_img_batch, xg_batch, y_batch in train_loader:
            x_img_batch = x_img_batch.to(device)
            xg_batch = xg_batch.to(device)
            y_batch = y_batch.to(device)

            batch_indices = (xg_batch[:, None] == x_graph_all).all(dim=2).nonzero(as_tuple=True)[1]
            outputs = model(x_img_batch, x_graph_all, edge_index_train, batch_indices)
            preds = outputs.argmax(dim=1)

            all_train_preds.extend(preds.cpu().numpy())
            all_train_labels.extend(y_batch.cpu().numpy())

    y_train_true = np.array(all_train_labels)
    y_train_pred = np.array(all_train_preds)

    train_oa = accuracy_score(y_train_true, y_train_pred)
    train_f1 = f1_score(y_train_true, y_train_pred, average='macro')
    train_kappa = cohen_kappa_score(y_train_true, y_train_pred)

    print(f"\nTraining Accuracy (OA): {train_oa:.4f}")
    print(f"Training F1 Score (Macro): {train_f1:.4f}")
    print(f"Training Cohen's Kappa: {train_kappa:.4f}")



Training Accuracy (OA): 0.9939
Training F1 Score (Macro): 0.9975
Training Cohen's Kappa: 0.9930
